<a href="https://colab.research.google.com/github/Tom-Jung/legendary-funicular/blob/main/Ch6_2.%20%EC%82%B0%EC%B6%9C%20%EB%B3%80%EB%8F%99%EC%9A%94%EC%9D%B8%20%EB%B6%84%ED%95%B4_3%EB%B6%80%EB%AC%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [485]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

In [486]:
np.set_printoptions(precision=3,suppress=True) # 결과를 소수 3자리까지만 보여주는 옵션

### 산출 변동요인분해

In [487]:
# 2015년 거래표 다운로드

url_total = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Total_Matrix_2015_cst.csv"  # 불변 2015년 총거래표
Tot_cst_15 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Imported_Matrix_2015_cst.csv"  # 불변 2015년 수입거래표
Imp_cst_15 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Domestic_Matrix_2015_cst.csv"  # 불변 2015년 국산거래표
Dom_cst_15 = pd.read_csv(url_dom)

In [488]:
# 2010년 거래표 다운로드

url_total = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Total_Matrix_2010_cst.csv"  # 불변 2010년 총거래표
Tot_cst_10 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Imported_Matrix_2010_cst.csv"  # 불변 2010년 수입거래표
Imp_cst_10 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Domestic_Matrix_2010_cst.csv"  # 불변 2010년 국산거래표
Dom_cst_10 = pd.read_csv(url_dom)

In [489]:
Tot_cst_10.shape

(41, 47)

부문통합하여 3부문 IO표 만들기

In [490]:
cdd = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/code.csv"  # 코드매치 파일
ccdd = pd.read_csv(cdd)
cd_for_itr = ccdd.to_numpy().reshape(33,2)
cd_3 = cd_for_itr[:,1]
cd_33 = cd_for_itr[:,0]
print('3부문 코드 : \n', cd_3)
print('33부문 코드 : \n', cd_33)

3부문 코드 : 
 [1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
33부문 코드 : 
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]


In [491]:
# Mtx = np.zeros((3,32)) # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽자 않으므로 반복문을 통해 3x33 임시코드 만들기
# row=cd_3[:-2]
# col=cd_33[:-2]
# for i, j in zip(row,col):
#   Mtx[i-1,j-1] = 1
# print(Mtx)

In [492]:
code = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],  # 행렬 곱을 이용한 부문 통합을 위해 0과 1로 구성된 임시의 3x33 매트릭스
       [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])
code = code.astype(float)

In [493]:
SS_t0 = (Tot_cst_10.iloc[1:41,2:].to_numpy().astype(float)) # 10년 총거래표 numpy로 숫자만 불러오기(시간 t-1)
Im_t0 = (Imp_cst_10.iloc[1:38,2:].to_numpy().astype(float)) # 10년 수입거래표 numpy로 숫자만 불러오기

SS_t5 = (Tot_cst_15.iloc[1:41,2:].to_numpy().astype(float)) # 15년 총거래표 numpy로 숫자만 불러오기(시간 t)
Im_t5 = (Imp_cst_15.iloc[1:38,2:].to_numpy().astype(float)) # 15년 수입거래표 numpy로 숫자만 불러오기


In [494]:
nae_tot_row_t0 = code@SS_t0[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기
nae_tot_row_t5 = code@SS_t5[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기

In [495]:
nae_tot_col_t0 = nae_tot_row_t0@code.T  # 중간투입(수요)의 열을 3부문으로 만들기
nae_tot_col_t5 = nae_tot_row_t5@code.T  # 중간투입(수요)의 열을 3부문으로 만들기

In [496]:
nae_tot_col_t5

array([[3.225e+06, 3.906e+07, 1.190e+07],
       [1.775e+07, 9.254e+08, 3.509e+08],
       [7.137e+06, 2.617e+08, 5.753e+08]])

In [497]:
va_row_t0 = SS_t0[32:42,0:32]@code.T # 중투합, 피용자보수, 영업잉여, 고정자본소모, 기타생산세, 조정항, 부가가치계, 산출액(8X3)
va_row_t5 = SS_t5[32:41,0:32]@code.T # 중투합, 피용자보수, 영업잉여, 고정자본소모, 기타생산세, 조정항, 부가가치계, 산출액(7X3)
va_row_t5

array([[2.811e+07, 1.226e+09, 9.381e+08],
       [5.243e+06, 1.768e+08, 5.682e+08],
       [2.258e+07, 1.400e+08, 2.789e+08],
       [4.828e+06, 1.148e+08, 1.900e+08],
       [6.100e+05, 5.344e+07, 8.205e+07],
       [3.326e+07, 4.850e+08, 1.119e+09],
       [6.137e+07, 1.711e+09, 2.057e+09]])

In [498]:
fd_cd = np.array([[1,1,0,0,0,0,0,0,0],   # 최종수요를 통합하기 위한 0과 1로된 행렬
                  [0,0,1,1,1,1,0,0,0],
                  [0,0,0,0,0,0,1,0,0],
                  [0,0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,0,0,0,1]])


In [499]:
fd_t0 = SS_t0[0:32, 33:42]
fd_t5 = SS_t5[0:32, 33:42]

In [500]:
fd_row_t0=code@fd_t0@fd_cd.T
fd_row_t5=code@fd_t5@fd_cd.T
fd_row_t0

array([[1.601e+07, 2.646e+06, 8.298e+05, 1.949e+07, 6.890e+07],
       [1.665e+08, 1.374e+08, 4.565e+08, 7.604e+08, 1.848e+09],
       [7.403e+08, 3.104e+08, 7.855e+07, 1.129e+09, 1.819e+09]])

In [501]:
fd_row_sm_t0 = np.sum((fd_row_t0), axis=0) # 최종수요의 행합
fd_row_sm_t5 = np.sum((fd_row_t5), axis=0) # 최종수요의 행합

In [502]:
fd_sm_t0=fd_row_sm_t0.reshape(1,5)
fd_sm_t5=fd_row_sm_t5.reshape(1,5)

In [503]:
nae_va_ct_t0 = np.concatenate((nae_tot_col_t0, va_row_t0)) # 중간투입(수요)와 부가가치 및 산출액 합치기
nae_va_ct_t5 = np.concatenate((nae_tot_col_t5, va_row_t5)) # 중간투입(수요)와 부가가치 및 산출액 합치기

In [504]:
nae_va_ct_sm_t0 = nae_va_ct_t0.sum(axis=1).reshape(11,1)  # 부가가치 및 산출액의 열합
nae_va_ct_sm_t5 = nae_va_ct_t5.sum(axis=1).reshape(10,1)  # 부가가치 및 산출액의 열합

In [505]:
nnull_t0 = np.zeros((7,5)) # 0행렬
nnull_t5 = np.zeros((6,5)) # 0행렬

In [506]:
fd_nnull_t0 = np.concatenate((fd_row_t0, fd_sm_t0, nnull_t0),axis=0)  # 최종수요, 최종수요 행합 및 0행렬 합치기
fd_nnull_t5 = np.concatenate((fd_row_t5, fd_sm_t5, nnull_t5),axis=0)  # 최종수요, 최종수요 행합 및 0행렬 합치기

In [507]:
tot_f_t0 = np.concatenate((nae_va_ct_t0,nae_va_ct_sm_t0, fd_nnull_t0), axis=1)  # 최종 3x3 부문통합된 거래표
tot_f_t5 = np.concatenate((nae_va_ct_t5,nae_va_ct_sm_t5, fd_nnull_t5), axis=1)  # 최종 3x3 부문통합된 거래표

In [508]:
tot_F_t0 = pd.DataFrame(tot_f_t0.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계', '피용자보수','영업잉여', '고정자본소모', '생산세(보조금공제)','조정항', '부가가치계', '총투입액']) # 변수명 붙여서 pandas로 내보내기


In [509]:
tot_F_t5 = pd.DataFrame(tot_f_t5.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계', '피용자보수','영업잉여', '고정자본소모', '생산세(보조금공제)','부가가치계', '총투입액']) # 변수명 붙여서 pandas로 내보내기


In [510]:
# 2010, 2015년 3부문 불변표 내보내기
tot_F_t0.to_csv('10년_총거래표_3부문_불변.csv', encoding="utf-8-sig")
tot_F_t5.to_csv('15년_총거래표_3부문_불변.csv', encoding="utf-8-sig")

### 수입거래표 3부문 통합

In [511]:
nae_imp_row_t0 = code@Im_t0[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기
nae_imp_row_t5 = code@Im_t5[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기

In [512]:
nae_imp_col_t0 = nae_imp_row_t0@code.T  # 중간투입(수요)의 열을 3부문으로 만들기
nae_imp_col_t5 = nae_imp_row_t5@code.T  # 중간투입(수요)의 열을 3부문으로 만들기

In [513]:
Im_t0.shape

(33, 42)

In [514]:
ism_row_t0 = (Im_t0[32,0:32]@code.T).reshape(1,3) # 중투합
ism_row_t5 = (Im_t5[32,0:32]@code.T).reshape(1,3) # 중투합
ism_row_t5

array([[1.881e+06, 3.520e+08, 1.233e+08]])

In [515]:
ifd_t0 = Im_t0[0:32, 33:42] #수입최종수요
ifd_t5 = Im_t5[0:32, 33:42] #수입최종수요

In [516]:
ifd_row_t0=code@ifd_t0@fd_cd.T
ifd_row_t5=code@ifd_t5@fd_cd.T

In [517]:
ifd_row_sm_t0 = np.sum((ifd_row_t0), axis=0) # 최종수요의 행합
ifd_row_sm_t5 = np.sum((ifd_row_t5), axis=0) # 최종수요의 행합

In [518]:
ifd_sm_t0=ifd_row_sm_t0.reshape(1,5)
ifd_sm_t5=ifd_row_sm_t5.reshape(1,5)

In [519]:
ism_row_t0.ndim

2

In [520]:
nae_im_t0 = np.concatenate((nae_imp_col_t0, ism_row_t0)) # 중간투입(수요)와 부가가치 및 산출액 합치기
nae_im_t5 = np.concatenate((nae_imp_col_t5, ism_row_t5)) # 중간투입(수요)와 부가가치 및 산출액 합치기

In [521]:
nae_col_sm_imp_t0 = nae_im_t0.sum(axis=1).reshape(4,1)  # 중간투입(수요)와 행합
nae_col_sm_imp_t5 = nae_im_t5.sum(axis=1).reshape(4,1)  # 중간투입(수요)와 행합

In [522]:
nae_im_sm_t0 = nae_col_sm_imp_t0.sum(axis=1).reshape(4,1)  # 중간투입 및 중간투입 열합
nae_im_sm_t5 = nae_col_sm_imp_t5.sum(axis=1).reshape(4,1)

In [523]:
ifd_nnull_t0 = np.concatenate((ifd_row_t0, ifd_sm_t0),axis=0)  #   최종수요 및 최종수요 행합 합치기
ifd_nnull_t5 = np.concatenate((ifd_row_t5, ifd_sm_t5),axis=0)  #   최종수요 및 최종수요 행합 합치기

In [524]:
imp_f_t0 = (np.concatenate((nae_im_t0, nae_im_sm_t0, ifd_nnull_t0), axis=1))  # 최종 3x3 부문통합된 거래표
imp_f_t5 = (np.concatenate((nae_im_t5, nae_im_sm_t5, ifd_nnull_t5), axis=1))  # 최종 3x3 부문통합된 거래표

In [525]:
imp_F_t0 = pd.DataFrame(imp_f_t0.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계']) # 변수명 붙여서 pandas로 내보내기
imp_F_t5 = pd.DataFrame(imp_f_t5.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계']) # 변수명 붙여서 pandas로 내보내기
imp_F_t5

,농산품,공산품,서비스,중간수요계,소비,투자,수출,최종수요계,총수요계
농산품,348818.0,8218892.0,748604.0,9316314.0,2859389.0,-17714.0,0.0,2841675.0,12157989.0
공산품,1157164.0,327866649.0,76354626.0,405378439.0,53140499.0,53398716.0,2125896.0,108665111.0,514043550.0
서비스,374663.0,15940859.0,46232222.0,62547744.0,28500448.0,6752632.0,0.0,35253080.0,97800824.0
중간투입계,1880645.0,352026400.0,123335452.0,477242497.0,84500336.0,60133634.0,2125896.0,146759866.0,624002363.0


In [526]:
# 2010, 2015년 3부문 불변표 내보내기
imp_F_t0.to_csv('10년_수입거래표_3부문_불변.csv', encoding="utf-8-sig")
imp_F_t5.to_csv('15년_수입거래표_3부문_불변.csv', encoding="utf-8-sig")

In [527]:
S1 = tot_f_t5.astype(float)
I1 = imp_f_t5.astype(float)
S0 = tot_f_t0.astype(float)
I0 = imp_f_t0.astype(float)
S1.sum()

40931720084.0

In [528]:
SS1 = np.concatenate((S1[:8,],np.zeros((1,9)), S1[8:10,] ), axis=0)  # 2015년 부가가치 조정항을 0으로 삽입(2010년과 동일한 포맷을 유지하기 위함)

In [529]:
SS0 = S0[0:3,0:3].astype(float)  # 내생항목 3x3
SS1 = S1[0:3,0:3].astype(float)
SS1.shape

(3, 3)

In [530]:
II0 = I0[0:3,0:3]  # 수입 내생항목 3x3
II1 = I1[0:3,0:3]

In [531]:
DD0 =SS0 - II0  # 국산 내생항목 3x3
DD1 =SS1 - II1
DD0

array([[4.129e+06, 2.900e+07, 8.603e+06],
       [1.676e+07, 5.103e+08, 2.352e+08],
       [5.354e+06, 2.091e+08, 4.273e+08]])

In [532]:
TM0 = I0[0:3,8]  # 총수입벡터
TM1 = I1[0:3,8]
TM0

array([9.798e+06, 4.169e+08, 7.243e+07])

In [533]:
IM0 = I0[0:3,3]  # 중간재수입벡터
IM1 = I1[0:3,3]
IM0

array([7.690e+06, 3.251e+08, 4.833e+07])

In [534]:
IF0 = I0[0:3,7]  # 최종재수입벡터
IF1 = I1[0:3,7]
IF0

array([ 2108171., 91865740., 24096241.])

In [535]:
SFC0 = S0[0:3,4]  # 최종수요-소비
SFC1 = S1[0:3,4]
SFC0

array([1.601e+07, 1.665e+08, 7.403e+08])

In [536]:
SFI0 = S0[0:3,5] # 최종수요-투자
SFI1 = S1[0:3,5]
SFI0/1000000

array([  2.646, 137.44 , 310.398])

In [537]:
SF0 = S0[0:3,7] # 최종수요합계
SF1 = S1[0:3,7]
SF0/1000000

array([  19.485,  760.437, 1129.235])

In [538]:
EX0 = S0[0:3,6] # 수출
EX1 = S1[0:3,6]
EX0/1000000

array([  0.83 , 456.483,  78.546])

In [539]:
DF0 = SF0 - EX0 #소비+투자
DF1 = SF1 - EX1
DF0/1000000

array([  18.656,  303.955, 1050.689])

In [540]:
SFC00 = SFC0.sum(axis=0) # 소비합계
SFC11 = SFC1.sum(axis=0)

SFI00 = SFI0.sum(axis=0) # 투자합계
SFI11 = SFI1.sum(axis=0)

EX00 = EX0.sum(axis=0) # 수출합계
EX11 = EX1.sum(axis=0)

print('2010년 소비합계 : ', SFC00)
print('2015년 소비합계 : ', SFC11)

print('2010년 투자합계 : ', SFI00)
print('2015년 투자합계 : ', SFI11)

print('2010년 수출합계 : ', EX00)
print('2015년 수출합계 : ', EX11)


2010년 소비합계 :  922814206.0
2015년 소비합계 :  1054900320.0
2010년 투자합계 :  450484823.0
2015년 투자합계 :  489601534.0
2010년 수출합계 :  535858572.0
2015년 수출합계 :  716951177.0


In [541]:
GDC1 = ((SFC11 - SFC00)/ SFC00) # 소비증가율
GDC2 = GDC1*SFC0  # 규모효과
GDC3 = (SFC1-SFC0) - GDC2 # 구조효과

In [542]:
GDI1 = ((SFI11 - SFI00)/ SFI00) # 투자증가율
GDI2 = GDI1*SFI0 # 규모효과
GDI3 = (SFI1-SFI0) - GDI2 # 구조효과

In [543]:
ED1 = ((EX11 - EX00)/EX00) # 수출증가율
ED2 = ED1*EX0 # 규모효과
ED3 = (EX1-EX0) - ED2 # 구조효과

In [544]:
C0 = S0[-1,0:3].T.reshape([3,1])  # 총산출액
C1 = S1[-1,0:3].T.reshape([3,1])
C0/1000000

array([[  59.105],
       [1430.838],
       [1746.898]])

In [545]:
TC0 = C0.sum(axis=0)  #총산출계
TC1 = C1.sum(axis=0)
TC0/1000000

array([3236.841])

In [546]:
V0 = S0[-1,0:3].T - S0[3,0:3].T # 부가가치계벡터 = 총투입액 - 중간투입계
V1 = S1[-1,0:3].T - S1[3,0:3].T
V1/1000000

array([  33.258,  485.04 , 1119.152])

In [547]:
AV0 = V0/C0 # 부가가치계수
AV1 = V1/C1
AV1

array([[ 0.542,  7.903, 18.236],
       [ 0.019,  0.283,  0.654],
       [ 0.016,  0.236,  0.544]])

In [548]:
C0

array([[5.911e+07],
       [1.431e+09],
       [1.747e+09]])

In [549]:
CT0 = np.tile(C0, reps=[1,3])   # Ct벡터를 반복해여 행렬(3x3)로 만듬
CT1 = np.tile(C1, reps=[1,3])
CT1

array([[6.137e+07, 6.137e+07, 6.137e+07],
       [1.711e+09, 1.711e+09, 1.711e+09],
       [2.057e+09, 2.057e+09, 2.057e+09]])

In [550]:
AS0 = SS0/CT0.T # 총투입계수표
AS1 = SS1/CT1.T
# np.isnan(AS_t)

In [551]:
AD0 = DD0/CT0.T # 국산투입계수표
AD1 = DD1/CT1.T

In [552]:
II0 = AS0-AD0 # 수입투입계수표
II1 = AS1-AD1

In [553]:
M0 = IF0/DF0 # 수입최종수요/(소비+투자)
M0 = np.diag(M0)

M1 = IF1/DF1
M1 = np.diag(M1)

In [554]:
RD0 = np.linalg.inv(np.eye(3) - AD0) # 생산유발계수
RD1 = np.linalg.inv(np.eye(3) - AD1)

In [555]:
AM0 = II0/AS0 #조정항목
AM0 = AM0*AS1

AM1 = II1/AS1
AM1 = AM1*AS0

# np.nan_to_num(AM0, copy=False)
# np.nan_to_num(AM1, copy=False)

In [556]:
AA = AS1 - AS0 #투입계수변화
AS1_f = pd.DataFrame(AS1)
AS1_f.to_csv('AS1_f.csv')

#### 산출액성장요인분해
-  파쉐방식으로 산출액 변동을 분해

In [557]:
TC1 = ((RD1@(np.eye(3)-M1))@GDC2).reshape([3,1])  # 소비규모증가효과
TC11 = ((RD1@(np.eye(3)-M1))@GDC3).reshape([3,1]) # 소비구조증가효과
TCTD = TC1 + TC11
TC111 = np.concatenate((TCTD, TC1, TC11), axis=1)
# TC111

In [558]:
TI1 = ((RD1@(np.eye(3)-M1))@GDI2).reshape([3,1])  # 소비규모증가효과
TI11 = ((RD1@(np.eye(3)-M1))@GDI3).reshape([3,1]) # 소비구조증가효과
TITD = TI1 + TI11
TI111 = np.concatenate((TITD, TI1, TI11), axis=1)

In [559]:
TQ = TC1+TI1 # 국내최종수요규모증가효과
TS = TC11+TI11 # 국내최종수요규조변화효과
TQS = TQ+TS
T1 = np.concatenate((TQS, TQ, TS), axis=1)
T1.shape

(3, 3)

In [560]:
T2 = (RD1@ED2).reshape([3,1]) # 수출규모증가효과
T22 = (RD1@ED3) .reshape([3,1])# 수출구조변화효과
TTE = T2+T22
T2 = np.concatenate((TTE, T2, T22), axis=1)

In [561]:
T3 = ((RD1@(M1 - M0))@DF0).reshape([3,1]) # 최종재수입대체효과
T3 = T3*-1
T4 = ((RD1@(AM1 - II0)))@C0[0:3,] # 중간재수입대체효과
T4 = T4*-1
T34 = T3 + T4
TI = np.concatenate((T34, T3, T4), axis=1)
TI

array([[ -2673230.026,  -1228385.444,  -1444844.583],
       [-45065548.09 , -20400323.626, -24665224.464],
       [-21648907.322, -13055064.519,  -8593842.803]])

In [562]:
T5 = (RD1@(AA-(II1-AM1)))@C0 # 기술변화효과
T5.shape
T5

array([[-2655781.777],
       [ 6683402.413],
       [30749629.274]])

In [563]:
DCT0 = C1-C0  #산출액 실질변동액
DCT1 = TCTD + TITD + TTE +T3 + T4 + T5 # 총변동요인
DCT = DCT0-DCT1

In [564]:
SUM = np.concatenate((C0, C1, DCT0, DCT1, DCT, T1, TC111, TI111, T2, TI, T5), axis=1)
SUM.shape
SUM1 = np.sum(SUM, axis=0).reshape([1,21])
PGD1015 = (np.concatenate((SUM, SUM1), axis=0))
PGD1015_f = (PGD1015/1000).round(1)
PGD1015_f
SUM/1000

array([[  59105.417,   61371.723,    2266.306,    2266.306,       0.   ,
           2392.746,    4777.142,   -2384.397,    2749.005,    3961.322,
          -1212.316,    -356.26 ,     815.821,   -1172.081,    5202.572,
           5645.583,    -443.011,   -2673.23 ,   -1228.385,   -1444.845,
          -2655.782],
       [1430838.343, 1711153.666,  280315.323,  280315.313,       0.01 ,
          61722.797,   77123.199,  -15400.403,   50074.173,   56604.89 ,
          -6530.717,   11648.623,   20518.309,   -8869.686,  256974.662,
         257074.324,     -99.662,  -45065.548,  -20400.324,  -24665.224,
           6683.402],
       [1746897.559, 2057212.517,  310314.958,  310314.955,       0.003,
         214444.021,  189315.923,   25128.098,  160878.466,  150005.238,
          10873.228,   53565.555,   39310.684,   14254.871,   86770.212,
          86269.517,     500.695,  -21648.907,  -13055.065,   -8593.843,
          30749.629]])

In [565]:
PGD1015_ff = pd.DataFrame(PGD1015_f,columns = ['10년산출액', '15년산출액', '15-10산출액(A)', '요인별합계(B)', 'B-A', '국내최종수요증감액', '국내최종수요(규모증가)', '국내최종수요(구조변화)',
                                                     '소비', '소비(규모)','소비(구조)','투자', '투자(규모)','투자(구조)', '수출', '수출(규모)','수출(구조)', '수입', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [566]:
PGD1015_ff

,10년산출액,15년산출액,15-10산출액(A),요인별합계(B),B-A,국내최종수요증감액,국내최종수요(규모증가),국내최종수요(구조변화),소비,소비(규모),...,투자,투자(규모),투자(구조),수출,수출(규모),수출(구조),수입,최종재수입,중간재수입,기술변화
농산품,59105.4,61371.7,2266.3,2266.3,0.0,2392.7,4777.1,-2384.4,2749.0,3961.3,...,-356.3,815.8,-1172.1,5202.6,5645.6,-443.0,-2673.2,-1228.4,-1444.8,-2655.8
공산품,1430838.3,1711153.7,280315.3,280315.3,0.0,61722.8,77123.2,-15400.4,50074.2,56604.9,...,11648.6,20518.3,-8869.7,256974.7,257074.3,-99.7,-45065.5,-20400.3,-24665.2,6683.4
서비스,1746897.6,2057212.5,310315.0,310315.0,0.0,214444.0,189315.9,25128.1,160878.5,150005.2,...,53565.6,39310.7,14254.9,86770.2,86269.5,500.7,-21648.9,-13055.1,-8593.8,30749.6
계,3236841.3,3829737.9,592896.6,592896.6,0.0,278559.6,271216.3,7343.3,213701.6,210571.5,...,64857.9,60644.8,4213.1,348947.4,348989.4,-42.0,-69387.7,-34683.8,-34703.9,34777.2


In [567]:
PGD1015_ff.to_csv('산출액변동요인.csv', encoding="utf-8-sig")

In [568]:
Ctbr =  PGD1015[:,5:]
Ctbr.shape

(4, 16)

In [569]:
sm_gap = PGD1015[:,3].reshape([4,1])  # 요인별 합계
sm_gap.shape

(4, 1)

In [570]:
sm_gap1 = np.tile(sm_gap, reps=[1,16]) # 요인별 합계를 33x16으로 반복
sm_gap1.shape

(4, 16)

In [571]:
Ctrb = (Ctbr/sm_gap1)*100  # 각 항목별 기여율
Ctrb_f = Ctrb.round(1)

In [572]:
Ctrb_ff = pd.DataFrame(Ctrb_f,columns = ['국내최종수요증감액', '국내최종수요(규모증가)', '국내최종수요(구조변화)','소비', '소비(규모)','소비(구조)','투자', '투자(규모)','투자(구조)',
                                         '수출', '수출(규모)','수출(구조)', '수입', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [573]:
Ctrb_ff.to_csv('항목별 기여율.csv', encoding="utf-8-sig")

In [575]:
YR = (((PGD1015[:,1]/PGD1015[:,0])**(1/5)-1)*100).reshape([4,1])
YR

array([[0.755],
       [3.643],
       [3.324],
       [3.421]])

In [576]:
Ctrb_amt = (Ctrb*YR/100)
Ctrb_amt_f = Ctrb_amt.round(3)
Ctrb_amt_f

array([[ 0.798,  1.592, -0.795,  0.916,  1.32 , -0.404, -0.119,  0.272,
        -0.391,  1.734,  1.882, -0.148, -0.891, -0.409, -0.482, -0.885],
       [ 0.802,  1.002, -0.2  ,  0.651,  0.736, -0.085,  0.151,  0.267,
        -0.115,  3.34 ,  3.341, -0.001, -0.586, -0.265, -0.321,  0.087],
       [ 2.297,  2.028,  0.269,  1.723,  1.607,  0.116,  0.574,  0.421,
         0.153,  0.93 ,  0.924,  0.005, -0.232, -0.14 , -0.092,  0.329],
       [ 1.607,  1.565,  0.042,  1.233,  1.215,  0.018,  0.374,  0.35 ,
         0.024,  2.014,  2.014, -0.   , -0.4  , -0.2  , -0.2  ,  0.201]])

In [577]:
Ctrb_amt_ff = pd.DataFrame(Ctrb_amt_f,columns = ['국내최종수요증감액', '국내최종수요(규모증가)', '국내최종수요(구조변화)','소비', '소비(규모)','소비(구조)','투자', '투자(규모)','투자(구조)',
                                         '수출', '수출(규모)','수출(구조)', '수입', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [578]:
Ctrb_amt_ff.to_csv('항목별 기여도.csv', encoding="utf-8-sig")